In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
from linearmodels.panel import RandomEffects, PanelOLS
import polars as pl
import numpy as np
from dotenv import load_dotenv
import os

load_dotenv()

az.style.use("arviz-darkgrid")
dr = DataReg()



In [ ]:
dr.notify(url=str(os.environ.get("URL")),auth=str(os.environ.get("AUTH")), msg="hi")

In [ ]:
dr.conn.sql("SELECT * FROM qcewtable WHERE ein=='660567825'").pl() #INFO We cound manualy obtain if they are foreing

In [ ]:
data = dr.regular_data(naics="21")
data

In [ ]:
data_pr = data[data["foreign"] == 0]
# data_pr = data_pr[data_pr["year"] > 2020]
model = bmb.Model(
    "log_total_employment ~ 0 + date + ein + log_k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results = model.fit()

In [ ]:

# results.posterior = results.posterior.rename({"k_index": "k_index_51"})
az.plot_trace(results)

In [ ]:
az.summary(results)

- rain by naics (2 digits)
- drop null zipcodes
- broke down the 72 (remove food services as its own sector)
- only the first 2 regression(0,1)


In [ ]:
naics_code = [
    # "11",
    "21",
    "22",
    # "23",
    # "31-33",
    # "42",
    # "44-45",
    # "48-49",
    # "51",
    # "52",
    # "54",
    # "55",
    # "56",
    # "61",
    # "62",
    # "71",
    # "72-accommodation",
    # "72-food",
    # "81",
    # "92"
]

In [ ]:
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning
)
warnings.filterwarnings(
    "ignore",
    category=UserWarning
)
for naics in naics_code:
    data = dr.regular_data(naics=naics)
    print(f"naics:{naics}")
    print(len(data))
    

In [ ]:
data_pr = data[data["foreign"] == 0]
model = bmb.Model(
    "log_total_employment ~ 0 + date + ein + log_k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler": "blackjax"},cores=10)



In [ ]:
az.plot_trace(results)

In [ ]:
data_pr = data[data["foreign"] == 1]
model = bmb.Model(
    "total_employment ~ 0 + date + ein + k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results2 = model.fit()

In [ ]:
az.to_netcdf(results2, "results_model2.nc")

In [ ]:
results2 = az.from_netcdf("results_model2.nc")

In [ ]:
combined_results = results.extend(results2)

In [ ]:
az.style.use("arviz-darkgrid")
# results.posterior = results.posterior.rename({"k_index": "k_index_51"})
az.plot_trace(results2)

In [ ]:


plt.savefig("data/results/trace_plot_pr.png", format="png")

res = az.summary(results)
res.to_csv("data/results/results_pr.csv")

In [ ]:
data_pr = data[data["foreign"] == 1]
model = bmb.Model(
    "total_employment ~ 0 + k_index + own_children6 + own_children17  + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler" :"blackjax"},cores=15)

az.plot_trace(
    results
)

plt.savefig("data/results/trace_plot_foreing.png", format="png")

res = az.summary(results)
res.to_csv("data/results/results_foreing.csv")

In [ ]:
model = bmb.Model(
    "total_employment ~ 0 + k_index + k_dummy + own_children6 + own_children17  + commute_car + food_stamp + with_social_security",
    data, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler" :"blackjax"},cores=15)

az.plot_trace(
    results
)

plt.savefig("data/results/trace_plot_full.png", format="png")

res = az.summary(results)
res.to_csv("data/results/results_full.csv")